In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## Funzione da ottimizzare TOROIDAL()

In [2]:
function toroidal(r=1., R=2., angle1=2*pi, angle2=2*pi)
    function toroidal0(shape=[24, 36])
        V, CV = Lar.simplexGrid(shape)
        V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let(u, v)=p;[(R+r*cos(u))*cos(v);
        (R+r*cos(u))*sin(v);-r*sin(u)]end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return toroidal0
end

toroidal (generic function with 5 methods)

In [3]:
@btime W,CW = Lar.toroidal()();

  4.157 ms (44198 allocations: 4.16 MiB)


In [4]:
@code_warntype Lar.toroidal()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#toroidal0#302"{Float64,Float64,Float64,Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(24, 36)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


In [5]:
@benchmark Lar.toroidal()()

BenchmarkTools.Trial: 964 samples with 1 evaluation.
 Range (min … max):  4.181 ms … 13.091 ms  ┊ GC (min … max): 0.00% … 39.42%
 Time  (median):     4.450 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.175 ms ±  1.421 ms  ┊ GC (mean ± σ):  7.18% ± 13.29%

  ██▆▅▄▂  ▂ ▂          ▂▃▁               ▂                    
  ███████████▇█▆▇▇██▇█▇████▆▇▅▄▆▅▅▅▄▄▄▄▇████▇▆█▅▄▄▄▁▄▅▁▄▆▆▄▅ █
  4.18 ms      Histogram: log(frequency) by time     9.91 ms <

 Memory estimate: 4.16 MiB, allocs estimate: 44196.

## Uso della macro @threads 

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

In [7]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

In [12]:
function toroidal2(r=1., R=2., angle1=2*pi, angle2=2*pi)
    function toroidal02(shape=[24, 36])
        V, CV = Lar.simplexGrid(shape)
        V = [angle1/(shape[1]) 0;0 angle2/(shape[2])]*V
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
        end 
        V = hcat(map(p->let(u, v)=p;[(R+r*cos(u))*cos(v);
        (R+r*cos(u))*sin(v);-r*sin(u)]end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return toroidal02
end

toroidal2 (generic function with 5 methods)

In [13]:
@btime W,CW = toroidal2()();

  4.178 ms (44613 allocations: 4.17 MiB)
